# Classificando complexidade de letras de músicas

### Critérios: 
Para analisar a dificuldade das letras, uma das abordagens seria analisar palavra por palavra da música e analisar: <br> <br>
1 - Número de sílabas. <br> 
. Palavras com diversas sílabas costumam ser mais complexas, tanto para compreensão quanto para a pronúncia. <br> <br>
2 - Diversidade semântica. <br> 
. Palavras com diversos significados podem ser mais difíceis, pois há uma gama de significados para elas <br> <br>
3 - Frequência. <br> 
4 - Uso de gírias e jargões <br> <br>
5 - Análise gramatical <br>
. Exemplo: frases que contêm muitos gerúndios são mais coloquiais e frases que possuem mais conjuções subordinadas são mais formais

### Número de sílabas:

Para descobrir o número de sílabas das letras das músicas, usarei a biblioteca Textstat. O objetivo é localizar quais as músicas que possuem os maiores números de sílabas. 

In [2]:
import textstat as tst
import pandas as pd

In [3]:
df = pd.read_csv("tcc_ceds_music.csv")

A textstat tem uma funcão chamada: "syllable_count()" que conta a quantidade de sílabas em uma palavra. Exemplo:

In [26]:
dic = {}

primeira_letra = df.lyrics[0] 
lista_palavras = primeira_letra.split(" ")
for palavra in lista_palavras:
    if palavra not in dic:
        dic[palavra] = tst.syllable_count(palavra)

In [27]:
primeira_letra

'hold time feel break feel untrue convince speak voice tear try hold hurt try forgive okay play break string feel heart want feel tell real truth hurt lie worse anymore little turn dust play house ruin run leave save like chase train late late tear try hold hurt try forgive okay play break string feel heart want feel tell real truth hurt lie worse anymore little run leave save like chase train know late late play break string feel heart want feel tell real truth hurt lie worse anymore little know little hold time feel'

Neste exemplo, peguei a primeira letra musical como exemplo. Para que cada palavra fosse lida individualmente, usei o split e apliquei em um for, adicionando-as em um dicionário.

#### Implementando para o projeto:

Para a implementação no projeto, criei um dicionário que tem como key o nome da música e como value a quantidade de sílabas que cada palavra da letra da música tem:

In [34]:
dic = {df.track_name[n]: {} for n in range(len(df))} 

for n in range(len(df)):
    lista = df.lyrics[n].split(" ")
    for palavra in lista:
        if palavra not in dic[df.track_name[n]]:
            dic[df.track_name[n]][palavra] = tst.syllable_count(palavra)

Com isso feito, buscaremos as palavras que contém mais de 5 sílabas e as adicionaremos em um dicionário:

In [35]:
musicas_silabas_complexas = {}

for nome_musica, palavras in dic.items():
    for palavra, silabas in palavras.items():
        if silabas >= 6:
            if not any(palavra.count(letra) >= 4 for letra in set(palavra)): 
                if nome_musica not in musicas_silabas_complexas:
                    musicas_silabas_complexas[nome_musica] = 1
                else:
                    musicas_silabas_complexas[nome_musica] += 1

Agora, adicionaremos a quantidade de palavras complexas que cada música tem no dataset:

In [120]:
df['contagem_palavras_complexas'] = df['track_name'].map(musicas_silabas_complexas).fillna(0)

#### Problema: 
Músicas com nomes iguais recebem o mesmo valor

In [125]:
df.loc[df["track_name"] == "the finer things"]["contagem_palavras_complexas"]

3309     2.0
19611    2.0
Name: contagem_palavras_complexas, dtype: float64

Para resolver, posso incluir o índice da música no dicionário para garantir que seja única, então corrigindo:

In [11]:
df.track_name = [f"{df.track_name[n]}_{n}" for n in range(len(df))]

In [37]:
df.track_name

0                     mohabbat bhi jhoothi_0
1                                i believe_1
2                                      cry_2
3                                 patricia_3
4                       apopse eida oneiro_4
                        ...                 
28367                  10 million ways_28367
28368    ante up (robbin hoodz theory)_28368
28369                    whutcha want?_28369
28370                           switch_28370
28371                           r.i.p._28371
Name: track_name, Length: 28372, dtype: object

In [105]:
dic = {df.track_name[n]: {} for n in range(len(df))} 

for n in range(len(df)):
    lista = df.lyrics[n].split(" ")
    for palavra in lista:
        if palavra not in dic[df.track_name[n]]:
            dic[df.track_name[n]][palavra] = tst.syllable_count(palavra)

musicas_silabas_complexas = {}

for nome_musica, palavras in dic.items():
    for palavra, silabas in palavras.items():
        if silabas >= 6:
            if not any(palavra.count(letra) >= 4 for letra in set(palavra)): 
                if nome_musica not in musicas_silabas_complexas:
                    musicas_silabas_complexas[nome_musica] = 1
                else:
                    musicas_silabas_complexas[nome_musica] += 1

In [106]:
df['silabas'] = df['track_name'].map(musicas_silabas_complexas).fillna(0)

### Frequência

Calcular o Type-Token Ratio (TTR) que é a razão entre o número de palavras únicas e o total

#### Type-Token Ratio (TTR)

In [32]:
palavras_unicas_list = []
lista_ttr = []

for letra in range(len(df)):
    palavras = df.lyrics[letra].lower().split()
    palavras_unicas_list.append(len(set(palavras)))
    
for n in range(len(palavras_unicas_list)):
    lista_ttr.append(palavras_unicas_list[n]/df.len[n])

### NLTK 

Funções da NLTK úteis para o projeto: <br> 
1 - Tokenização: Divide o texto em tokens para facilitar a análise <br>
2 - Análise sintática: Realizar a análise gramatical de sentenças <br>
3 - Tagging: Identifica substantivos, verbos, conjunções, ... 


Primeiros usos:

In [33]:
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize

In [36]:
texto = "micael é muito"

Separando em sentenças:

In [41]:
sentenca = sent_tokenize(texto)
sentenca

['micael é muito']

Separando em tokens:

In [43]:
tokens = word_tokenize(texto)
tokens

['micael', 'é', 'muito']

Separando em tags: <br>
Para usar a função pos_tag, é necessário passar como parâmetro a frase separada em tokens

In [46]:
nltk.pos_tag(tokens)

[('micael', 'NN'), ('é', 'NNP'), ('muito', 'NN')]

O PorterStemmer mostra a raiz das palavras:

In [47]:
from nltk.stem import PorterStemmer

In [49]:
stemmer = PorterStemmer()

In [52]:
palavras = ["running", "ran", "easily", "fairly"]
stems = [stemmer.stem(palavra) for palavra in palavras]
stems

['run', 'ran', 'easili', 'fairli']

* Para buscar palavras com muitos sinônimos:

In [54]:
from nltk.corpus import wordnet

In [76]:
sinonimos = wordnet.synsets("happy")

In [77]:
sinonimos_unicos = set()
for syn in sinonimos:
    for lemma in syn.lemmas():
        sinonimos_unicos.add(lemma.name())

In [78]:
sinonimos_unicos

{'felicitous', 'glad', 'happy', 'well-chosen'}

Analisar a quantidade de hiponimos: <br>
obs: Hipônimos são palavras que têm um significado mais específico do que outras, ou seja, são termos que se referem a algo de forma mais detalhada. 


In [83]:
hiponimos = []
for syn in wordnet.synsets("animal"):
    hiponimos.extend(syn.hyponyms())

Analisar a quantidade de hiperônimos: <br>
OBS: Hiperônimo é uma palavra que tem um significado mais abrangente do que outras palavras que estão no mesmo campo semântico

In [86]:
hiperonimos = []
for syn in wordnet.synsets("animal"):
    hiperonimos.extend(syn.hypernyms())

Analise de coocorrência de palavras:

In [102]:
from nltk import bigrams
from collections import Counter

In [89]:
bi_grams = list(bigrams(texto))

In [90]:
bi_grams

[('m', 'i'),
 ('i', 'c'),
 ('c', 'a'),
 ('a', 'e'),
 ('e', 'l'),
 ('l', ' '),
 (' ', 'é'),
 ('é', ' '),
 (' ', 'm'),
 ('m', 'u'),
 ('u', 'i'),
 ('i', 't'),
 ('t', 'o')]

In [128]:
textos = [
    "Eu tenho um cachorro que adora brincar com meu gato.",
    "O gato é muito independente e gosta de dormir com cachorro.",
    "Meu cachorro e meu gato são melhores amigos.",
    "Os cachorros gostam de passear no parque.",
    "Gatos são muito diferentes de cachorros em muitos aspectos."
]

In [129]:
tokens = []
for texto in textos:
    tokens.extend(word_tokenize(texto.lower()))

In [130]:
bi_grams = list(bigrams(tokens))

In [131]:
co = Counter(bi_grams)

In [134]:
concorrencia_gato_cachorro = co[('cachorro', 'gato')] + co[('gato', 'cachorro')]

In [135]:
concorrencia_gato_cachorro

0

### Análise gramatical

NNP: Proper Noun, Singular (substantivo próprio, singular)

NNPS: Proper Noun, Plural (substantivo próprio, plural)

JJ: Adjective (adjetivo)

JJR: Adjective, Comparative (adjetivo, comparativo - ex: "better", "larger")

JJS: Adjective, Superlative (adjetivo, superlativo - ex: "best", "largest")

RB: Adverb (advérbio)

RBR: Adverb, Comparative (advérbio, comparativo - ex: "better", "faster")

RBS: Adverb, Superlative (advérbio, superlativo - ex: "best", "fastest")

VB: Verb, Base Form (verbo, forma base - ex: "run", "play")

VBD: Verb, Past Tense (verbo, passado - ex: "ran", "played")

VBG: Verb, Gerund or Present Participle (verbo, gerúndio ou particípio presente - ex: "running", "playing")

VBN: Verb, Past Participle (verbo, particípio passado - ex: "run", "played")

VBP: Verb, Non-3rd Person Singular Present (verbo, presente, não 3ª pessoa singular - ex: "run", "play")

VBZ: Verb, 3rd Person Singular Present (verbo, presente, 3ª pessoa singular - ex: "runs", "plays")

WDT: Wh-Determiner (determinante interrogativo - ex: "which", "that")

IN: Preposition or Subordinating Conjunction (preposição ou conjunção subordinativa - ex: "in", "on", "because")

In [23]:
dic_tags = {track: 0 for track in df['track_name']}

for index, row in df.iterrows():
    tokens = word_tokenize(row['lyrics'].lower())
    tags = nltk.pos_tag(tokens)
    
    pontuacao = 0
    for palavra, tag in tags:

        if tag in ['NNP', 'NNPS']:
            pontuacao += 2
        elif tag in ['JJ', 'JJR', 'JJS']:
            pontuacao += 1
        elif tag in ['RB', 'RBR', 'RBS']:
            pontuacao += 1
        elif tag in ['VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ']:
            pontuacao += 1
        elif tag == 'WDT':
            pontuacao += 1
        elif tag == 'IN':
            pontuacao += 1
            
    dic_tags[row['track_name']] = pontuacao

### Gensim

Um dos usos possíveis dessa ferramente é medir a proximidade semântica entre as palavras e verificar se a música explora temas variados ou complexos

In [33]:
from gensim.models import Word2Vec

In [34]:
sentences = [
    ['o', 'gato', 'está', 'no', 'telhado'],
    ['o', 'cachorro', 'está', 'na', 'rua'],
    ['eu', 'amo', 'programar', 'em', 'Python'],
    ['a', 'cachorra', 'e', 'o', 'gato', 'são', 'amigos'],
    ['Python', 'é', 'uma', 'linguagem', 'de', 'programação'],
]

In [35]:
model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)

In [36]:
word_vector = model.wv['gato']

In [39]:
similar_words = model.wv.most_similar('gato', topn=5)
similar_words

[('em', 0.18859755992889404),
 ('a', 0.160723015666008),
 ('amigos', 0.15932446718215942),
 ('de', 0.1372387409210205),
 ('cachorra', 0.12300693243741989)]